# Lecture 6 - Valuation (Python practice)

This notebook focuses on the **Python implementation** of the core valuation toolkit used in Lecture 6.

We keep the model intentionally compact for teaching: a simple revenue forecast -> free cash flow -> DCF, plus an
illustrative **carbon-cost adjustment** scenario.


## Setup: data + paths

The next code cell prepares everything the DCF section depends on:

1. Imports packages (`numpy`, `pandas`).
2. Sets `DATA_DIR` and `SLIDE_DIR` so the notebook works whether it is run from the repo root or from `materials/`.
3. Loads Tesla financial statements from local Excel files.
4. Defines `base_year` as the most recent year in the dataset.

Required data files (in `materials/data/`):
- `tesla_income_stmt.xlsx`
- `tesla_cash_flow.xlsx`
- `tesla_balance_sheet.xlsx`

A daily price file `tesla_daily_price.xlsx` is used later to build a simple market-cap proxy for WACC weights.


In [1]:
# Setup: imports + local data paths
from pathlib import Path
import os

import numpy as np
import pandas as pd

# Optional online fetches (akshare) are disabled by default.
RUN_ONLINE = False
RUN_ONLINE = RUN_ONLINE or (os.environ.get('RUN_ONLINE', '0') == '1')

# Make paths robust whether Jupyter is launched from repo root or from `materials/`.
DATA_DIR = Path('data') if Path('data').exists() else Path('materials/data')
SLIDE_DIR = (
    Path('pic') / '0209_lecture6_firm_valuation'
    if (Path('pic') / '0209_lecture6_firm_valuation').exists()
    else Path('materials/pic/0209_lecture6_firm_valuation')
)

if not DATA_DIR.exists():
    raise FileNotFoundError(f'Cannot find data directory: {DATA_DIR.resolve()}')
if not SLIDE_DIR.exists():
    raise FileNotFoundError(f'Cannot find slide directory: {SLIDE_DIR.resolve()}')

# Load Tesla statements used in the DCF exercise.
income = pd.read_excel(DATA_DIR / 'tesla_income_stmt.xlsx', index_col=0)
cashflow = pd.read_excel(DATA_DIR / 'tesla_cash_flow.xlsx', index_col=0)
balance = pd.read_excel(DATA_DIR / 'tesla_balance_sheet.xlsx', index_col=0)

base_year = income.index.max()
prev_year = income.index.sort_values()[-2]
base_year


Timestamp('2025-12-31 00:00:00')

## DCF helper functions

The next code cell implements two reusable formulas used throughout the valuation exercise:

- **Present value of a cash-flow stream** (discounted at WACC)
  $$PV = \sum_{t=1}^{n} \frac{CF_t}{(1+WACC)^t}$$

- **Terminal value (growing perpetuity)**
  $$TV_n = \frac{FCF_n(1+g)}{WACC-g} \quad \text{(requires } g < WACC\text{)}$$


In [2]:
def dcf_pv(cashflows, wacc):
    cashflows = np.asarray(cashflows, dtype=float)
    t = np.arange(1, len(cashflows) + 1)
    return (cashflows / (1 + wacc) ** t).sum()

def terminal_value_growing_perpetuity(fcf_t, wacc, g):
    if g >= wacc:
        raise ValueError('g must be smaller than WACC for a stable perpetuity.')
    return fcf_t * (1 + g) / (wacc - g)


### Python practice: a compact Tesla DCF + carbon adjustment

We now implement a *teaching* DCF model that follows the slide logic:

- forecast revenue, EBIT margin, D&A/revenue, capex/revenue
- estimate operating working capital from incremental revenue
- discount by WACC and compute a growing-perpetuity terminal value

Then we connect carbon footprint -> valuation by subtracting an after-tax **carbon cost** scenario.


<details>
<summary>revenue</summary>

![](pic\revenue.png)

</details>


In [3]:
# Base-year ratios for forecasting
tax_rate = float(income.loc[base_year, 'Tax Rate For Calcs'])
rev0 = float(income.loc[base_year, 'Total Revenue'])
ebit0 = float(income.loc[base_year, 'EBIT'])
da0 = float(cashflow.loc[base_year, 'Depreciation'])
capex0 = float(cashflow.loc[base_year, 'Capital Expenditure'])

# Working capital in the FCF formula typically refers to *operating* NWC,
# excluding cash and financing items.
current_assets = float(balance.loc[base_year, 'Current Assets'])
current_liabilities = float(balance.loc[base_year, 'Current Liabilities'])
cash_like = float(balance.loc[base_year, 'Cash Cash Equivalents And Short Term Investments'])
current_debt = float(balance.loc[base_year, 'Current Debt'])
op_wc0 = (current_assets - cash_like) - (current_liabilities - current_debt)

ebit_margin = ebit0 / rev0
da_ratio = da0 / rev0
capex_ratio = (-capex0) / rev0
op_wc_ratio = op_wc0 / rev0

# Simple revenue-growth assumptions (edit freely)
# Yahoo Finance analyst forecasts (provided): 2026=8.72%, 2027=19.11%
rev_growth_2026 = 0.0872  # 8.72%
rev_growth_2027 = 0.1911  # 19.11%
rev_growth_long_run = (rev_growth_2026 + rev_growth_2027) / 2
forecast_years = [2026, 2027, 2028, 2029, 2030]
revenue_growth = [rev_growth_2026, rev_growth_2027, rev_growth_long_run, rev_growth_long_run, rev_growth_long_run]

revenue = [rev0]
for g in revenue_growth:
    revenue.append(revenue[-1] * (1 + g))
revenue = np.array(revenue[1:])

proj = pd.DataFrame({'Year': forecast_years, 'Revenue': revenue})
proj['EBIT'] = proj['Revenue'] * ebit_margin
proj['NOPAT'] = proj['EBIT'] * (1 - tax_rate)
proj['D&A'] = proj['Revenue'] * da_ratio
proj['CapEx'] = proj['Revenue'] * capex_ratio
proj['ΔWC'] = (proj['Revenue'].diff().fillna(proj['Revenue'].iloc[0] - rev0)) * op_wc_ratio
proj['FCF'] = proj['NOPAT'] + proj['D&A'] - proj['CapEx'] - proj['ΔWC']

proj_view = proj.assign(
    **{
        'Revenue ($B)': proj['Revenue'] / 1e9,
        'ΔWC ($B)': proj['ΔWC'] / 1e9,
        'FCF ($B)': proj['FCF'] / 1e9,
    }
)[['Year', 'Revenue ($B)', 'ΔWC ($B)', 'FCF ($B)']]
proj_view.round(2)


,Year,Revenue ($B),ΔWC ($B),FCF ($B)
0,2026,103.10,-0.49,2.36
1,2027,122.80,-1.16,3.38
2,2028,139.88,-1.00,3.54
3,2029,159.35,-1.14,4.03
4,2030,181.52,-1.30,4.59


In [4]:
# WACC (teaching assumptions)
price = pd.read_excel(DATA_DIR / 'tesla_daily_price.xlsx', index_col=0)
latest_close = float(price['Close'].iloc[-1])
shares = float(balance.loc[base_year, 'Ordinary Shares Number'])
market_cap = shares * latest_close

total_debt = float(balance.loc[base_year, 'Total Debt'])
cash_eq = float(balance.loc[base_year, 'Cash And Cash Equivalents'])
net_debt = total_debt - cash_eq
V = market_cap + total_debt

# Note on inputs:
# - rf: U.S. 10-year Treasury yield (DGS10)
# - ERP: average excess return of the S&P 500 over the risk-free rate (teaching assumption here)
# - rd: estimated from outstanding corporate bonds (market-value-weighted average) (teaching assumption here)

rf10 = pd.read_csv(DATA_DIR / 'us_10y_treasury_yield_dgs10.csv')
rf10['Date'] = pd.to_datetime(rf10['Date'])
rf10 = rf10.set_index('Date').sort_index()
rf = float(rf10['DGS10'].ffill().iloc[-1]) / 100  # annualized (decimal)

beta = 1.89  # Yahoo Finance estimate (provided)
erp = 0.10   # equity risk premium (provided)
re = rf + beta * erp

rd_pre_tax = 0.06  # placeholder; in practice use MV-weighted yield on outstanding corporate bonds
rd = rd_pre_tax * (1 - tax_rate)

wacc = re * (market_cap / V) + rd * (total_debt / V)

wacc


0.22918245021187825

In [5]:
# Terminal growth (cap at ~4%)
equity = balance['Common Stock Equity'].dropna().sort_index()
ni = income['Net Income From Continuing Operation Net Minority Interest'].dropna().sort_index()
roe_2025 = float(ni.loc[base_year] / ((equity.loc[base_year] + equity.loc[prev_year]) / 2))
g_terminal = float(np.clip(roe_2025, 0.0, 0.04))

g_terminal


0.04

In [6]:
# Baseline valuation
fcf = proj['FCF'].to_numpy()
tv = terminal_value_growing_perpetuity(fcf_t=fcf[-1], wacc=wacc, g=g_terminal)
enterprise_value = dcf_pv(fcf, wacc) + tv / (1 + wacc) ** len(fcf)
equity_value = enterprise_value - net_debt
intrinsic_price = equity_value / shares

{'EV': enterprise_value, 'Equity value': equity_value, 'USD/share': intrinsic_price}


{'EV': np.float64(18467844081.347923),
 'Equity value': np.float64(20261844081.347923),
 'USD/share': np.float64(5.401717963569161)}

In [7]:
# Carbon adjustment (illustrative): carbon cost = (revenue * intensity) * carbon price
ei = 12.0  # tCO2e per $1M revenue (Scope 1+2 proxy)
carbon_price = 100.0  # $/tCO2e

revenue_musd = proj['Revenue'] / 1e6
carbon_cost = revenue_musd * ei * carbon_price
fcf_carbon = fcf - carbon_cost.to_numpy() * (1 - tax_rate)

enterprise_value_carbon = dcf_pv(fcf_carbon, wacc) + terminal_value_growing_perpetuity(
    fcf_t=fcf_carbon[-1], wacc=wacc, g=g_terminal
) / (1 + wacc) ** len(fcf_carbon)
equity_value_carbon = enterprise_value_carbon - net_debt
price_carbon = equity_value_carbon / shares

{'USD/share (baseline)': intrinsic_price, 'USD/share (carbon-adjusted)': price_carbon}


{'USD/share (baseline)': np.float64(5.401717963569161),
 'USD/share (carbon-adjusted)': np.float64(5.23109498411108)}

## Page 23 - Sensitivity Analysis

Sensitivity analysis asks: how much does intrinsic value change when we vary key assumptions such as:

- WACC
- terminal growth rate $g$
- (optionally) carbon price / emissions intensity

Use the slide screenshot for the sensitivity table layout.

<details>
<summary>Slide screenshot (PDF page 23)</summary>

![](pic/0209_lecture6_firm_valuation/page-23.png)

</details>


## Page 24 - Types of multiples

Common valuation multiples and when they are useful:

| Multiple | When it is useful |
|---|---|
| Enterprise value / sales (EV/Sales) | Helpful for high-growth or low-profit firms |
| Enterprise value / EBITDA (EV/EBITDA) | Popular in cyclical industries; cross-country comparisons; less sensitive to leverage |
| Enterprise value / EBIT (EV/EBIT) | Useful for capital-intensive businesses |
| Price / earnings (P/E) | Sensitive to one-offs; forward P/E is widely used |
| Price / book (P/B) | Often used for financial institutions |
| Enterprise value / total assets (EV/Assets) | Useful for utilities and other fixed-asset-heavy firms |

*Source in slide: J.P. Morgan.*

<details>
<summary>Slide screenshot (PDF page 24)</summary>

![](pic/0209_lecture6_firm_valuation/page-24.png)

</details>


## Page 25 - Tesla Comparable Firms

The slide provides a peer set for **pure-play EV manufacturers**. Use it as a starting point for comparable-company
analysis (see screenshot).

<details>
<summary>Slide screenshot (PDF page 25)</summary>

![](pic/0209_lecture6_firm_valuation/page-25.png)

</details>


## Page 26 - Tesla Comparable Firms

The slide provides a peer set for **technology / autonomy-adjacent firms**. These peers can be useful when you
want to cross-check EV makers against broader tech valuations (see screenshot).

<details>
<summary>Slide screenshot (PDF page 26)</summary>

![](pic/0209_lecture6_firm_valuation/page-26.png)

</details>


## Page 27 - Comparable Company Analysis

Comparable-company analysis typically:

1. selects a peer set
2. computes valuation multiples (EV/Sales, EV/EBITDA, P/E, etc.)
3. applies peer multiples to the target's metrics to infer an implied valuation range

See the slide screenshot for the template.

<details>
<summary>Slide screenshot (PDF page 27)</summary>

![](pic/0209_lecture6_firm_valuation/page-27.png)

</details>


### Comparable-company practice (optional)

The slide suggests replacing the template with Tesla's comparables.
Below is a compact `akshare` workflow for a handful of peer valuation metrics.

Tip: this section is **optional** and may require network access. Set `RUN_ONLINE = True` (or export `RUN_ONLINE=1`)
if you want to fetch live market data.


In [8]:
try:
    import akshare as ak
except ModuleNotFoundError:
    ak = None
    print('akshare is not installed; skipping comparable-company practice (optional).')

import time

PEERS = ['TSLA', 'GM', 'F', 'TM', 'NVDA', 'AAPL', 'GOOGL']

CACHE_PATH = DATA_DIR / 'peer_snapshot.csv'

# Baidu valuation indicators (ak.stock_us_valuation_baidu)
IND_MCAP = '总市值'
IND_PE = '市盈率(TTM)'
IND_PB = '市净率'
IND_PCF = '市现率'
PERIOD = '近一年'
ANNUAL = '年报'

def with_retry(fn, retries=3, backoff=0.6):
    last = None
    for k in range(retries):
        try:
            return fn()
        except Exception as e:
            last = e
            time.sleep(backoff * (k + 1))
    raise last

def latest_value(df):
    # akshare returns columns: date, value
    return float(df.iloc[-1]['value'])

if ak is None:
    RUN_ONLINE = False

rows = []
peer_snapshot = pd.DataFrame()
if RUN_ONLINE:
    for tkr in PEERS:
        try:
            mcap_yi = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_MCAP, PERIOD)))
            pe_ttm = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PE, PERIOD)))
            pb = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PB, PERIOD)))
            pcf = latest_value(with_retry(lambda: ak.stock_us_valuation_baidu(tkr, IND_PCF, PERIOD)))

            # Unit note: Baidu reports 总市值 in '亿' (1e8). Convert to USD dollars.
            market_cap = mcap_yi * 1e8

            # Latest close from Sina (ak.stock_us_daily)
            px = with_retry(lambda: ak.stock_us_daily(symbol=tkr, adjust=''))
            close = float(px.iloc[-1]['close'])

            # Revenue (latest annual) from Eastmoney indicator table
            fin = with_retry(lambda: ak.stock_financial_us_analysis_indicator_em(symbol=tkr, indicator=ANNUAL))
            fin_latest = fin.sort_values('REPORT_DATE').iloc[-1]
            revenue = float(fin_latest['OPERATE_INCOME'])
            currency = str(fin_latest.get('CURRENCY_ABBR', ''))

            mcap_to_rev = (market_cap / revenue) if currency == 'USD' and revenue else np.nan

            rows.append(
                {
                    'Ticker': tkr,
                    'Close': close,
                    'Market cap ($)': market_cap,
                    'P/E (TTM)': pe_ttm,
                    'P/B': pb,
                    'P/CF': pcf,
                    'Revenue (latest)': revenue,
                    'Currency': currency,
                    'MktCap/Revenue (USD only)': mcap_to_rev,
                }
            )
        except Exception as e:
            rows.append({'Ticker': tkr, 'Error': f'{type(e).__name__}: {e}'})
    peer_snapshot = pd.DataFrame(rows).set_index('Ticker') if rows else pd.DataFrame()
    if not peer_snapshot.empty:
        peer_snapshot.to_csv(CACHE_PATH)
        print(f'Cached peer snapshot to: {CACHE_PATH.resolve()}')
else:
    if CACHE_PATH.exists():
        peer_snapshot = pd.read_csv(CACHE_PATH, index_col=0)
        print(f'Loaded cached peer snapshot from: {CACHE_PATH.resolve()}')
    else:
        print('Skipping akshare fetch (RUN_ONLINE=False) and no cache found.')

peer_snapshot


Loaded cached peer snapshot from: C:\Users\Jinquan Ye\OneDrive - Duke University\Research\teaching\K10_calculation\materials\data\peer_snapshot.csv


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),Currency,MktCap/Revenue (USD only)
Ticker,,,,,,,,
TSLA,411.11,1.542662e+12,406.60,18.77,98.35,9.482700e+10,USD,16.268173
GM,84.24,7.615000e+10,28.24,1.25,3.05,1.850190e+11,USD,0.411579
F,13.80,5.498600e+10,11.68,1.22,2.73,1.849920e+11,USD,0.297234
TM,244.22,3.183010e+11,12.71,1.28,10.23,4.803670e+13,JPY,NaN
NVDA,185.41,4.505463e+12,45.42,37.90,54.59,1.304970e+11,USD,34.525414
AAPL,278.12,4.083119e+12,34.67,46.37,30.41,4.161610e+11,USD,9.811393
GOOGL,322.86,3.905637e+12,29.55,9.40,23.97,4.028360e+11,USD,9.695352


In [9]:
from IPython.display import display

if peer_snapshot.empty or peer_snapshot.dropna(how='all').empty:
    print('No online data to summarize. Set RUN_ONLINE=True to fetch via akshare.')
else:
    numeric = peer_snapshot.select_dtypes(include='number')
    if numeric.empty:
        print('No numeric columns to summarize (all fetches failed).')
    else:
        summary = numeric.describe(percentiles=[0.5]).loc[['mean', '50%']]
        display(summary)


,Close,Market cap ($),P/E (TTM),P/B,P/CF,Revenue (latest),MktCap/Revenue (USD only)
mean,219.965714,2.069474e+12,81.267143,16.598571,31.904286,7.064434e+12,11.834858
50%,244.220000,1.542662e+12,29.550000,9.400000,23.970000,1.850190e+11,9.753373
